In [8]:
import torch
import pickle
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import ast

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### 生成文本表示

In [9]:
# 需要使用代理才能正常下载
from transformers import BertTokenizer, BertModel
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese').cuda()

In [ ]:
# 读loaded_data取保存的 CSV 文件
loaded_data = pd.read_csv('..\data\selected_book_top_1200_data_tag.csv')

loaded_data['Tags'] = loaded_data['Tags'].apply(lambda x: ast.literal_eval(x))

In [11]:
tag_embedding_dict = {}

with torch.no_grad():
    for index, rows in tqdm(loaded_data.iterrows()):
        # 将标签列表转换为字符串
        tags_str = " ".join(rows.Tags)
        # 使用BERT中文模型对标签进行编码
        inputs = tokenizer(tags_str, truncation=True, return_tensors='pt')
        outputs = model(inputs.input_ids.cuda(), inputs.token_type_ids.cuda(), inputs.attention_mask.cuda())
        # 使用最后一层的平均隐藏状态作为标签的向量表示
        tag_embedding = outputs.last_hidden_state.mean(dim=1).cpu()
        tag_embedding_dict[rows.Book] = tag_embedding


1200it [00:53, 22.62it/s]


In [12]:
import pickle

# 将映射表存储为二进制文件
with open('data/tag_embedding_dict.pkl', 'wb') as f:
    pickle.dump(tag_embedding_dict, f)


# 如果已经进行上面的步骤 再次打开时可直接从下面开始

In [ ]:
import torch
import pickle
import numpy as np
import pandas as pd
from torch import nn
from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import ndcg_score
import ast

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [2]:
# 从二进制文件中读取映射表
with open('data/tag_embedding_dict.pkl', 'rb') as f:
    tag_embedding_dict = pickle.load(f)

In [3]:
# 读loaded_data取保存的 CSV 文件
loaded_data = pd.read_csv('../data/book_score.csv')

# 显示加载的数据
print(loaded_data)

           User     Book  Rate                       Time         Tag
0       1398478  1467022     0  2011-03-29T12:48:35+08:00         NaN
1       1398478  1777823     0  2011-02-02T21:58:55+08:00         NaN
2       1398478  1902628     0  2011-01-31T15:57:58+08:00         NaN
3       1398478  1878708     0  2011-01-26T11:27:59+08:00         NaN
4       1398478  4238362     0  2011-01-21T13:04:15+08:00         NaN
...         ...      ...   ...                        ...         ...
637249  4507957  1125186     4  2009-07-04T08:02:13+08:00  张爱玲,半生缘,爱情
637250  4507957  1002299     5  2009-07-04T08:01:28+08:00  金庸,武侠,笑傲江湖
637251  4507957  1001136     4  2009-07-04T07:55:17+08:00     彼得・潘,童话
637252  4507957  1021615     5  2009-07-04T07:53:54+08:00   小王子,童话,经典
637253  4507957  1962929     5  2009-06-29T22:13:37+08:00          爱情

[637254 rows x 5 columns]


In [4]:
class BookRatingDataset(Dataset):
    def __init__(self, data, user_to_idx, book_to_idx, tag_embedding_dict):
        self.data = data
        self.user_to_idx = user_to_idx
        self.book_to_idx = book_to_idx
        self.tag_embedding_dict = tag_embedding_dict

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        user = self.user_to_idx[row['User']]
        book = self.book_to_idx[row['Book']]
        rating = row['Rate'].astype('float32')
        text_embedding = self.tag_embedding_dict.get(row['Book'])
        return user, book, rating, text_embedding

class MatrixFactorization(nn.Module):
    def __init__(self, num_users, num_books, embedding_dim, hidden_state):
        super(MatrixFactorization, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim)
        self.book_embeddings = nn.Embedding(num_books, embedding_dim)
        self.linear_embedding = nn.Linear(hidden_state, embedding_dim)
        self.output = nn.Linear(embedding_dim, 6)

    def forward(self, user, book, tag_embedding):
        user_embedding = self.user_embeddings(user)
        book_embedding = self.book_embeddings(book)
        tag_embedding_proj = self.linear_embedding(tag_embedding)
        book_intergrate = book_embedding + tag_embedding_proj
        return (user_embedding * book_intergrate).sum(dim = 1)
        
def create_id_mapping(id_list):
    # 从ID列表中删除重复项并创建一个排序的列表
    unique_ids = sorted(set(id_list))
    
    # 创建将原始ID映射到连续索引的字典
    id_to_idx = {id: idx for idx, id in enumerate(unique_ids)}
    
    # 创建将连续索引映射回原始ID的字典
    idx_to_id = {idx: id for id, idx in id_to_idx.items()}
    
    return id_to_idx, idx_to_id

# 按用户分组计算NDCG
def compute_ndcg(group):
    true_ratings = group['true'].tolist()
    pred_ratings = group['pred'].tolist()
    return ndcg_score([true_ratings], [pred_ratings], k = 50)

In [10]:
user_ids = loaded_data['User'].unique()
book_ids = loaded_data['Book'].unique()

user_to_idx, idx_to_user = create_id_mapping(user_ids)
book_to_idx, idx_to_book = create_id_mapping(book_ids)

# 划分训练集和测试集
train_data, test_data = train_test_split(loaded_data, test_size=0.5, random_state=42)

# 创建训练集和测试集的数据集对象
train_dataset = BookRatingDataset(train_data, user_to_idx, book_to_idx, tag_embedding_dict)
test_dataset = BookRatingDataset(test_data, user_to_idx, book_to_idx, tag_embedding_dict)

# 创建训练集和测试集的数据加载器
train_dataloader = DataLoader(train_dataset, batch_size=4096, shuffle=True, drop_last = True)
test_dataloader = DataLoader(test_dataset, batch_size=4096, shuffle=False, drop_last = True)

num_users = loaded_data['User'].nunique()  
num_books = loaded_data['Book'].nunique() 
embedding_dim, hidden_state = 32, 768

model = MatrixFactorization(num_users, num_books, embedding_dim, hidden_state).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### 训练

In [ ]:
num_epochs = 20
lambda_u, lambda_b = 0.001, 0.001

for epoch in range(num_epochs):
    model.train()
    total_loss_train, total_loss_test = 0.0, 0.0

    for idx, (user_ids, book_ids, ratings, tag_embedding) in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch {epoch}'):
        # 使用user_ids, book_ids, ratings进行训练

        optimizer.zero_grad()
        
        predictions = model(user_ids.to(device), book_ids.to(device), tag_embedding.squeeze(1).to(device))
        loss = criterion(predictions, ratings.to(device)) + lambda_u * model.user_embeddings.weight.norm(2) + lambda_b * model.book_embeddings.weight.norm(2)
        
        loss.backward()
        optimizer.step()

        total_loss_train += loss.item()
        
        # if idx % 100 == 0:
        #     print(f'第{epoch+1}轮，第{idx}步，训练损失：{loss.item()}')

    output_loss_train = total_loss_train / (idx + 1) 
    print(f'第{epoch+1}轮，平均训练损失：{output_loss_train}')

    results = []
    model.eval()

    with torch.no_grad():
        for idx, (user_ids, item_ids, true_ratings, tag_embedding) in enumerate(test_dataloader):
            pred_ratings = model(user_ids.to(device), item_ids.to(device), tag_embedding.squeeze(1).to(device))

            loss = criterion(pred_ratings, ratings.to(device))
            total_loss_test += loss.item()

            # 将结果转换为 numpy arrays
            user_ids_np = user_ids.long().cpu().numpy().reshape(-1, 1)
            pred_ratings_np = pred_ratings.cpu().numpy().reshape(-1, 1)
            true_ratings_np = true_ratings.numpy().reshape(-1, 1)

            # 将这三个 arrays 合并成一个 2D array
            batch_results = np.column_stack((user_ids_np, pred_ratings_np, true_ratings_np))

            # 将这个 2D array 添加到 results
            results.append(batch_results)

        # 将结果的 list 转换为一个大的 numpy array
        results = np.vstack(results)

        # 将结果转换为DataFrame
        results_df = pd.DataFrame(results, columns=['user', 'pred', 'true'])
        results_df['user'] = results_df['user'].astype(int)

        ndcg_scores = results_df.groupby('user').apply(compute_ndcg)

        # 计算平均NDCG
        avg_ndcg = ndcg_scores.mean()
        print(f'Epoch {epoch}, Train loss: {output_loss_train}, Test loss:, {total_loss_test / (idx + 1)}, Average NDCG: {avg_ndcg}')

1it [00:00,  1.90it/s]

第1轮，第0步，训练损失：45.015174865722656


77it [00:31,  2.46it/s]


第1轮，平均训练损失：42.668808726521284
第1轮，测试集第0步，预测完成
第1轮，测试集第1步，预测完成
第1轮，测试集第2步，预测完成
第1轮，测试集第3步，预测完成
第1轮，测试集第4步，预测完成
第1轮，测试集第5步，预测完成
第1轮，测试集第6步，预测完成
第1轮，测试集第7步，预测完成
第1轮，测试集第8步，预测完成
第1轮，测试集第9步，预测完成
第1轮，测试集第10步，预测完成
第1轮，测试集第11步，预测完成
第1轮，测试集第12步，预测完成
第1轮，测试集第13步，预测完成
第1轮，测试集第14步，预测完成
第1轮，测试集第15步，预测完成
第1轮，测试集第16步，预测完成
第1轮，测试集第17步，预测完成
第1轮，测试集第18步，预测完成
第1轮，测试集第19步，预测完成
第1轮，测试集第20步，预测完成
第1轮，测试集第21步，预测完成
第1轮，测试集第22步，预测完成
第1轮，测试集第23步，预测完成
第1轮，测试集第24步，预测完成
第1轮，测试集第25步，预测完成
第1轮，测试集第26步，预测完成
第1轮，测试集第27步，预测完成
第1轮，测试集第28步，预测完成
第1轮，测试集第29步，预测完成
第1轮，测试集第30步，预测完成
第1轮，测试集第31步，预测完成
第1轮，测试集第32步，预测完成
第1轮，测试集第33步，预测完成
第1轮，测试集第34步，预测完成
第1轮，测试集第35步，预测完成
第1轮，测试集第36步，预测完成
第1轮，测试集第37步，预测完成
第1轮，测试集第38步，预测完成
第1轮，测试集第39步，预测完成
第1轮，测试集第40步，预测完成
第1轮，测试集第41步，预测完成
第1轮，测试集第42步，预测完成
第1轮，测试集第43步，预测完成
第1轮，测试集第44步，预测完成
第1轮，测试集第45步，预测完成
第1轮，测试集第46步，预测完成
第1轮，测试集第47步，预测完成
第1轮，测试集第48步，预测完成
第1轮，测试集第49步，预测完成
第1轮，测试集第50步，预测完成
第1轮，测试集第51步，预测完成
第1轮，测试集第52步，预测完成
第1轮，测试集第53步，预测完成
第1轮，测试集第54步，预测完成
第1轮，测试集第55步，预测完成
第1轮，测试集第56步，预测完成
第1轮，测试集第57步

C:\Users\22609\AppData\Local\Temp\ipykernel_19980\1417372104.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = results_df.groupby('user').apply(compute_ndcg)


...

1it [00:00,  2.94it/s]

第20轮，第0步，训练损失：1.9295698404312134


77it [00:39,  1.94it/s]


第20轮，平均训练损失：2.1173517363412038
第20轮，测试集第0步，预测完成
第20轮，测试集第1步，预测完成
第20轮，测试集第2步，预测完成
第20轮，测试集第3步，预测完成
第20轮，测试集第4步，预测完成
第20轮，测试集第5步，预测完成
第20轮，测试集第6步，预测完成
第20轮，测试集第7步，预测完成
第20轮，测试集第8步，预测完成
第20轮，测试集第9步，预测完成
第20轮，测试集第10步，预测完成
第20轮，测试集第11步，预测完成
第20轮，测试集第12步，预测完成
第20轮，测试集第13步，预测完成
第20轮，测试集第14步，预测完成
第20轮，测试集第15步，预测完成
第20轮，测试集第16步，预测完成
第20轮，测试集第17步，预测完成
第20轮，测试集第18步，预测完成
第20轮，测试集第19步，预测完成
第20轮，测试集第20步，预测完成
第20轮，测试集第21步，预测完成
第20轮，测试集第22步，预测完成
第20轮，测试集第23步，预测完成
第20轮，测试集第24步，预测完成
第20轮，测试集第25步，预测完成
第20轮，测试集第26步，预测完成
第20轮，测试集第27步，预测完成
第20轮，测试集第28步，预测完成
第20轮，测试集第29步，预测完成
第20轮，测试集第30步，预测完成
第20轮，测试集第31步，预测完成
第20轮，测试集第32步，预测完成
第20轮，测试集第33步，预测完成
第20轮，测试集第34步，预测完成
第20轮，测试集第35步，预测完成
第20轮，测试集第36步，预测完成
第20轮，测试集第37步，预测完成
第20轮，测试集第38步，预测完成
第20轮，测试集第39步，预测完成
第20轮，测试集第40步，预测完成
第20轮，测试集第41步，预测完成
第20轮，测试集第42步，预测完成
第20轮，测试集第43步，预测完成
第20轮，测试集第44步，预测完成
第20轮，测试集第45步，预测完成
第20轮，测试集第46步，预测完成
第20轮，测试集第47步，预测完成
第20轮，测试集第48步，预测完成
第20轮，测试集第49步，预测完成
第20轮，测试集第50步，预测完成
第20轮，测试集第51步，预测完成
第20轮，测试集第52步，预测完成
第20轮，测试集第53步，预测完成
第20轮，测试

C:\Users\22609\AppData\Local\Temp\ipykernel_19980\1417372104.py:57: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ndcg_scores = results_df.groupby('user').apply(compute_ndcg)
